In [ ]:
import torch
import os
import sys
import random

# custom
from util import *
from AAC_Prefix.AAC_Prefix import * # network
from Train import *
    
TEST_BATCH_SIZE = 5

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [ ]:
# table_num = 1 : Evaluation on Clotho
# table_num = 2 : Evaluation on AudioCaps

# setting_num = 1 : train dataset == test dataset
# setting_num = 2 : train dataset != test dataset
# setting_num = 3 : overall datasets(Clotho & AudioCaps) <- need to test by using compressed audio

table_num = 2
setting_num = 2

if setting_num == 3 :
    is_settingnum_3 = True
else : 
    is_settingnum_3 = False

model = get_model_in_table(table_num, setting_num, device)

In [ ]:
test_dataloader_audiocaps  = CreateDataloader(None, './AudioCaps', TEST_BATCH_SIZE, 'test', None, is_TrainDataset = False, tokenizer_type = None, is_settingnum_3 = is_settingnum_3)
test_dataloader_clotho = CreateDataloader(None, './Clotho', TEST_BATCH_SIZE, 'evaluation', None, is_TrainDataset = False, tokenizer_type = None, is_settingnum_3 = is_settingnum_3)

In [ ]:
metrics, captions_pred, captions_gt = eval_model(model, test_dataloader_clotho, 31, 'test', True, device, 'Clotho')

In [ ]:
metrics, captions_pred, captions_gt = eval_model(model, test_dataloader_audiocaps, 31, 'test', True, device, 'AudioCaps')